In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# Load data
train_images = np.load('/content/train_images.npy').reshape(-1, 28*28)
train_labels = np.load('/content/train_labels.npy').squeeze(1)
val_images = np.load('/content/val_images.npy').reshape(-1, 28*28)
val_labels = np.load('/content/val_labels.npy').squeeze(1)
test_images = np.load('/content/test_images.npy').reshape(-1, 28*28)
test_labels = np.load('/content/test_labels.npy').squeeze(1)

# Convert data to PyTorch tensors
train_images = torch.tensor(train_images, dtype=torch.float32)
train_labels = torch.tensor(train_labels, dtype=torch.long)
val_images = torch.tensor(val_images, dtype=torch.float32)
val_labels = torch.tensor(val_labels, dtype=torch.long)
test_images = torch.tensor(test_images, dtype=torch.float32)
test_labels = torch.tensor(test_labels, dtype=torch.long)

# Define neural network model
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(28*28, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model, loss function, and optimizer
model = NeuralNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Check the sizes of the images and labels arrays
print("Number of samples in train images:", len(train_images))
print("Number of samples in train labels:", len(train_labels))

# Downsample the images array to match the size of the labels array
train_images = train_images[:len(train_labels)]

# Shuffle both the training images and labels arrays using the same set of indices
indices = np.random.permutation(len(train_images))
train_images = train_images[indices]
train_labels = train_labels[indices]

# Training loop
for epoch in range(10):
    model.train()
    optimizer.zero_grad()
    outputs = model(train_images)
    loss = criterion(outputs, train_labels)
    loss.backward()
    optimizer.step()

# Evaluate on test set
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    correct = 0
    total = 0
    batch_size = 100
    num_batches = len(test_images) // batch_size  # Calculate number of full batches
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size
        images = test_images[start_idx:end_idx]
        labels = test_labels[start_idx:end_idx]
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Handle the remaining samples
    if len(test_images) % batch_size != 0:
        start_idx = num_batches * batch_size
        images = test_images[start_idx:]
        labels = test_labels[start_idx:]
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Calculate accuracy
    accuracy = correct / total
    print(f'Accuracy on test set: {accuracy:.4f}')


# Plot 4 random images from the test set with prediction and true label
num_samples = 4
random_indices = np.random.choice(len(test_images), num_samples, replace=False)
for idx in random_indices:
    image = test_images[idx].numpy().reshape(28, 28)
    label = test_labels[idx].item()
    output = model(test_images[idx].unsqueeze(0))
    _, predicted = torch.max(output, 1)
    predicted_label = predicted.item()

    plt.imshow(image, cmap='gray')
    plt.title(f'Pred: {predicted_label}, True: {label}')
    plt.axis('off')
    plt.show()



Number of samples in train images: 269988
Number of samples in train labels: 89996
